<a href="https://colab.research.google.com/github/imymemineyay/MBTI/blob/main/%EC%8B%A0%ED%95%9C_%EC%86%8CBTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. 라이브러리 불러오기**

In [ ]:
pip install xlwt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 5.3 MB/s eta 0:00:00


In [ ]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import re
import xlwt

## **2. 데이터 불러오기**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data1 = pd.read_csv('/content/drive/MyDrive/Shinhan_AI_Comp/Data/ gender_consumption_pattern.csv', encoding='cp949') # encoding='cp949' 한글 전용 인코딩
data2 = pd.read_csv('/content/drive/MyDrive/Shinhan_AI_Comp/Data/time_consumption_pattern.csv', encoding='cp949')
code = pd.read_excel('/content/drive/MyDrive/Shinhan_AI_Comp/Data/code.xls')

## **3. 데이터 확인 및 데이터 수정**

In [ ]:
# 코드 데이터 확인
code.head(3)

,업종코드(UPJONG_CD),대분류(CLASS1),중분류(CLASS2),소분류(CLASS3),SUM 하고 전국 평균과 비교 후 E/I로 구별,CASE 1 분류,이유,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,ss001,요식/유흥,한식,한식,외식,P,외식 = 호캉스 나 맛집에 해당하기 때문에 P,"여행, 캠핑, 항공업종",E,I,"배달, 구독, 인테리어"
1,ss002,요식/유흥,일식/중식/양식,일식,외식,P,외식 = 호캉스 나 맛집에 해당하기 때문에 P,온라인 소비,N,S,체험형 소비
2,ss003,요식/유흥,일식/중식/양식,양식,외식,P,외식 = 호캉스 나 맛집에 해당하기 때문에 P,내외면 성장,T,F,기부나 전기차


In [ ]:
# 업종코드와 case 1 분류(업종코드별 8유형부여한 분류코드)
using_code = code.loc[:,['업종코드(UPJONG_CD)','CASE 1 분류']]
using_code.columns = ['upjong_cd','classification']
using_code['upjong_cd'] = using_code['upjong_cd'].str.replace(r'[a-zA-Z]+', lambda x: x.group().upper()) # 대문자로 바꿔주기
using_code.head(5)

<ipython-input-74-095427bcbb9c>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  using_code['upjong_cd'] = using_code['upjong_cd'].str.replace(r'[a-zA-Z]+', lambda x: x.group().upper()) # 대문자로 바꿔주기


,upjong_cd,classification
0,SS001,P
1,SS002,P
2,SS003,P
3,SS004,P
4,SS005,P


In [ ]:
# 성별연령대별 카드소비 패턴.csv 확인
data1.head(3)

,서울시민업종코드(UPJONG_CD),기준년월(YM),고객주소블록코드(BLOCK_CD),성별(GEDNER),연령대별(AGE),카드이용금액계(AMT_CORR),카드이용건수계(USECT_CORR)
0,SS013,201906,14037,F,50대,48288,65
1,SS048,201608,156830,M,30대,132792,10
2,SS016,202009,32925,M,50대,100600,25


In [ ]:
# 필요한 데이터 : UPJONG_CD, AMT_CORR

using_data1 = data1.loc[:,['서울시민업종코드(UPJONG_CD)','카드이용금액계(AMT_CORR)']]
using_data1.columns = ['upjong_cd','amt_corr']
using_data1.head(3)

,upjong_cd,amt_corr
0,SS013,48288
1,SS048,132792
2,SS016,100600


In [ ]:
# 날짜별 시간대별 카드소비 패턴.csv 확인
data2.head(3)

,서울시민업종코드(UPJONG_CD),기준년월(YM),시간대구간(TIME),고객주소블록코드(BLOCK_CD),카드이용금액계(AMT_CORR),소액결제건수(MICRO_PYM)
0,SS017,202006,6,11063,257,25
1,SS013,201608,6,216137,86,10
2,SS016,201802,3,224819,35,15


In [ ]:
# 필요한 데이터 : UPJONG_CD, AMT_CORR

using_data2 = data2.loc[:,['서울시민업종코드(UPJONG_CD)','카드이용금액계(AMT_CORR)']]
using_data2.columns = ['upjong_cd','amt_corr']
using_data2.head(3)

,upjong_cd,amt_corr
0,SS017,257
1,SS013,86
2,SS016,35


In [ ]:
# using_data1 과 using_data2 합병 
consumer = pd.concat([using_data1,using_data2])
consumer

,upjong_cd,amt_corr
0,SS013,48288
1,SS048,132792
2,SS016,100600
3,SS001,213876
4,SS044,155930
...,...,...
495,SS017,5
496,SS016,10
497,SS081,5
498,SS069,70


**💙 사용할 데이터 프레임 : code, consumer**

#### **code, consumner 결합하기**

## **4. 최종 데이터 만들기**

In [ ]:
completed_df = pd.merge(consumer, using_code, left_on='upjong_cd', right_on='upjong_cd', how='inner')
completed_df = completed_df[['upjong_cd','classification','amt_corr']] # 컬럼 순서 변경
completed_df

,upjong_cd,classification,amt_corr
0,SS013,S/E,48288
1,SS013,S/E,2663385
2,SS013,S/E,960730
3,SS013,S/E,566831
4,SS013,S/E,422520
...,...,...,...
995,SS081,T,5
996,SS092,T,4395969
997,SS050,T,5
998,SS059,I,5


#### 🚨🚨 **불필요한 행 삭제**

SS040, SS045, SS046, SS048, SS049, SS054, SS055, SS056, SS057, SS058, SS059, SS060, SS063, SS064, SS066 

교통, 세탁소, 업무서비스, 통신, 보험, 의료(대분류), 가전, 가구, 자동차판매, 자동차 서비스, 주유소 등은 **mbti 유형 구분이 힘들다고 판단하여 삭제 결정**

In [ ]:
remove_list = ['SS040', 'SS045', 'SS046', 'SS048', 'SS049', 'SS054', 'SS055', 'SS056', 'SS057', 'SS058', 'SS059', 'SS060', 'SS063', 'SS064', 'SS066']
completed_df = completed_df[~completed_df['upjong_cd'].isin(remove_list)]

## **5. 35명의 가상인물 데이터프레임 생성**


각 데이터프레임 명칭이랑 사용할 난수

    id071 517, id072 20256, id073 3942, id074 345983, id075 23432, id076 564, 
    id077 234, id078 7274, id079 4756, id080 2342, id081 156246, id082 854, 
    id083 525, id084 685, id085 679, id086 57, id087 267, id088 8765, id089 38,
    id090 15, id091 114, id092 1973, id093 8472, id094 4759, id095 3302, 
    id096 5840, id097 10485, id098 93745, id099 3728, id100 5990, id101 22321, 
    id102 22222, id103 33332, id104 9990, id105 79987

**DataFrame에서 무작위로 몇 개의 값을 출력하는 함수**

```
DataFrame.sample(n=None, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)
```


**매개변수 설명**

    n : 추출할 개수. frac 매개변수와 동시에 사용할 수 없다.
    frac : 추출할 비율. 1 이하의 값을 설정하며 n과 동시에 사용할 수 없다.
    replace : 중복 샘플링 허용 여부.
    weight : 샘플링할 데이터와 동일한 길이의 Series를 샘플링에 사용할 가중치로 전달한다.
    random_state : seed를 설정할 수 있다. 임의 값을 지정하면 항상 같은 값을 반환한다.
    axis : 추출할 축을 지정한다.
    ignore_index : True이면 반환 데이터프레임 인덱스를 초기화한다.

#### 👉🏼 **35개의 무작위 난수를 이용하여 중복 허용하고 100개의 데이터를 추출할 것** 

In [ ]:
id071 = completed_df.sample(100, replace=True, random_state = 517) # no 1.
id072 = completed_df.sample(100, replace=True, random_state = 20256) # no 2.
id073 = completed_df.sample(100, replace=True, random_state = 3942) # no 3.
id074 = completed_df.sample(100, replace=True, random_state = 345983) # no 4.
id075 = completed_df.sample(100, replace=True, random_state = 23432) # no 5.
id076 = completed_df.sample(100, replace=True, random_state = 564) # no 6.
id077 = completed_df.sample(100, replace=True, random_state = 234) # no 7.
id078 = completed_df.sample(100, replace=True, random_state = 7274) # no 8.
id079 = completed_df.sample(100, replace=True, random_state = 4756) # no 9.
id080 = completed_df.sample(100, replace=True, random_state = 2342) # no 10.
id081 = completed_df.sample(100, replace=True, random_state = 156246) # no 11.
id082 = completed_df.sample(100, replace=True, random_state = 854) # no 12.
id083 = completed_df.sample(100, replace=True, random_state = 525) # no 13.
id084 = completed_df.sample(100, replace=True, random_state = 685) # no 14.
id085 = completed_df.sample(100, replace=True, random_state = 679) # no 15.
id086 = completed_df.sample(100, replace=True, random_state = 57) # no 16.
id087 = completed_df.sample(100, replace=True, random_state = 267) # no 17.
id088 = completed_df.sample(100, replace=True, random_state = 8765) # no 18.
id089 = completed_df.sample(100, replace=True, random_state = 38) # no 19.
id090 = completed_df.sample(100, replace=True, random_state = 15) # no 20.
id091 = completed_df.sample(100, replace=True, random_state = 114) # no 21.
id092 = completed_df.sample(100, replace=True, random_state = 1973) # no 22.
id093 = completed_df.sample(100, replace=True, random_state = 8472) # no 23.
id094 = completed_df.sample(100, replace=True, random_state = 4759) # no 24.
id095 = completed_df.sample(100, replace=True, random_state = 3302) # no 25.
id096 = completed_df.sample(100, replace=True, random_state = 5840) # no 26.
id097 = completed_df.sample(100, replace=True, random_state = 10485) # no 27.
id098 = completed_df.sample(100, replace=True, random_state = 93745) # no 28.
id099 = completed_df.sample(100, replace=True, random_state = 3728) # no 29.
id100 = completed_df.sample(100, replace=True, random_state = 5990) # no 30.
id101 = completed_df.sample(100, replace=True, random_state = 22321) # no 31.
id102 = completed_df.sample(100, replace=True, random_state = 22222) # no 32.
id103 = completed_df.sample(100, replace=True, random_state = 33332) # no 33.
id104 = completed_df.sample(100, replace=True, random_state = 9990) # no 34.
id105 = completed_df.sample(100, replace=True, random_state = 79987) # no 35.


## **6. 텍스트 빈도수 구하기** 

##### **str.isalnum()메서드**

: 문자가 영숫자 문자인 경우, True 반환 → 즉, 문자열에 특수문자가 없음을 의미 

: 문자열에 특수 문자가 있으면 False 반환



```
string = "Hey! What's up bro?"

new_string = ''.join(char for char in string if char.isalnum())
print(new_string)
```


##### **filter(str.isalnum, string)메서드**

```
string = "Hey! What's up bro?"

new_string = ''.join(filter(str.isalnum, string)) 
print(new_string)
```


##### **정규식**

: 문자열에서 특수 문자를 제거하기 위해 문자열에서 특수 문자를 자동으로 제거하는 정규식 작성 가능
→ [^a-zA-Z0-9]

: ^는 괄호 안의 문자를 제외한 모든 문자를 나타내고 a-zA-Z0-9는 문자열만 가질 수 있음을 나타냄 
소문자 및 대문자 알파벳과 숫자


```
import re

string = "Hey! What's up bro?"
new_string = re.sub(r"[^a-zA-Z0-9]","",string)
print(new_string)
```



출처 : https://www.delftstack.com/ko/howto/python/remove-special-characters-from-string-python/

참고 : https://wikidocs.net/4308


In [ ]:
def reg(name):
    mbti = name['classification'].to_list()
    mbti = [i.replace('/','') for i in mbti]
    so_bti = []
    if mbti.count('E') > mbti.count('I') :
        so_bti.append('E')
    else:
        so_bti.append('I')

    if mbti.count('N') > mbti.count('S') :
        so_bti.append('N')
    else:
        so_bti.append('S')

    if  mbti.count('T') > mbti.count('F') :
        so_bti.append('T')
    else:
        so_bti.append('F')

    if  mbti.count('P') > mbti.count('J') :
        so_bti.append('P')
    else:
        so_bti.append('J')
    result = so_bti[0] + so_bti[1] + so_bti[2] + so_bti[3]
    return result

In [ ]:
id071['mbti'] = reg(id071)
id072['mbti'] = reg(id072)
id073['mbti'] = reg(id073)
id074['mbti'] = reg(id074)
id075['mbti'] = reg(id075)
id076['mbti'] = reg(id076)
id077['mbti'] = reg(id077)
id078['mbti'] = reg(id078)
id079['mbti'] = reg(id079)
id080['mbti'] = reg(id080)
id081['mbti'] = reg(id081)
id082['mbti'] = reg(id082)
id083['mbti'] = reg(id083)
id084['mbti'] = reg(id084)
id085['mbti'] = reg(id085)
id086['mbti'] = reg(id086)
id087['mbti'] = reg(id087)
id088['mbti'] = reg(id088)
id089['mbti'] = reg(id089)
id090['mbti'] = reg(id090)
id091['mbti'] = reg(id091)
id092['mbti'] = reg(id092)
id093['mbti'] = reg(id093)
id094['mbti'] = reg(id094)
id095['mbti'] = reg(id095)
id096['mbti'] = reg(id096)
id097['mbti'] = reg(id097)
id098['mbti'] = reg(id098)
id099['mbti'] = reg(id099)
id100['mbti'] = reg(id100)
id101['mbti'] = reg(id101)
id102['mbti'] = reg(id102)
id103['mbti'] = reg(id103)
id104['mbti'] = reg(id104)
id105['mbti'] = reg(id105)

## **7. upjong_cd의 SS 제거 후 내보내기**

In [ ]:
def remove_ss(id):
    id['upjong_cd'] = id['upjong_cd'].str.replace(r'[a-zA-Z]+', lambda x: '')
    del id['classification']

In [ ]:
remove_ss(id071)
remove_ss(id072)
remove_ss(id073)
remove_ss(id074)
remove_ss(id075)
remove_ss(id076)
remove_ss(id077)
remove_ss(id078)
remove_ss(id079)
remove_ss(id080)
remove_ss(id081)
remove_ss(id082)
remove_ss(id083)
remove_ss(id084)
remove_ss(id085)
remove_ss(id086)
remove_ss(id087)
remove_ss(id088)
remove_ss(id089)
remove_ss(id090)
remove_ss(id091)
remove_ss(id092)
remove_ss(id093)
remove_ss(id094)
remove_ss(id095)
remove_ss(id096)
remove_ss(id097)
remove_ss(id098)
remove_ss(id099)
remove_ss(id100)
remove_ss(id101)
remove_ss(id102)
remove_ss(id103)
remove_ss(id104)
remove_ss(id105)

<ipython-input-110-a5e77b071085>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  id['upjong_cd'] = id['upjong_cd'].str.replace(r'[a-zA-Z]+', lambda x: '')


In [ ]:
id071['id'] = 'id071'
id072['id'] = 'id072'
id073['id'] = 'id073'
id074['id'] = 'id074'
id075['id'] = 'id075'
id076['id'] = 'id076'
id077['id'] = 'id077'
id078['id'] = 'id078'
id079['id'] = 'id079'
id080['id'] = 'id080'
id081['id'] = 'id081'
id082['id'] = 'id082'
id083['id'] = 'id083'
id084['id'] = 'id084'
id085['id'] = 'id085'
id086['id'] = 'id086'
id087['id'] = 'id087'
id088['id'] = 'id088'
id089['id'] = 'id089'
id090['id'] = 'id090'
id091['id'] = 'id091'
id092['id'] = 'id092'
id093['id'] = 'id093'
id094['id'] = 'id094'
id095['id'] = 'id095'
id096['id'] = 'id096'
id097['id'] = 'id097'
id098['id'] = 'id098'
id099['id'] = 'id099'
id100['id'] = 'id100'
id101['id'] = 'id101'
id102['id'] = 'id102'
id103['id'] = 'id103'
id104['id'] = 'id104'
id105['id'] = 'id105'

In [ ]:
id71_105 = pd.concat([id071,id072,id073,id074,id075,id076,id077,id078,id079,id080,id081,id082,id083,id084,id085,id086,id087,id088,id089,id090,id091,
                      id092,id093,id094,id095,id096,id097,id098,id099,id100,id101,id102,id103,id104,id105])
id71_105 = id71_105[['id', 'upjong_cd','amt_corr', 'mbti']] # 데이터프레임 순서 변경 id - upjong_cd - amt_corr - mbti)
id71_105

,upjong_cd,amt_corr,mbti,id
33,013,45,ISTP,id071
668,021,25,ISTP,id071
611,006,80,ISTP,id071
390,069,5,ISTP,id071
923,015,5,ISTP,id071
...,...,...,...,...
123,016,10,ISTP,id105
321,022,17605,ISTP,id105
385,069,15090,ISTP,id105
475,004,504439,ISTP,id105


In [ ]:
output_path = '/content/drive/MyDrive/Shinhan_AI_Comp/Data/id71_105.xls'
id71_105.to_excel(output_path, index=False)

<ipython-input-123-24063747e854>:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  id71_105.to_excel(output_path, index=False)
